In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 전처리 정의

In [2]:
import tqdm
import torch
import torch.nn as nn

from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor
from torchvision.transforms import RandomHorizontalFlip, RandomCrop
from torchvision.transforms import Normalize
from torch.utils.data.dataloader import DataLoader
from torchvision.models.resnet import resnet34, resnet18


from torch.optim.adam import Adam

# 학습할 때 이용할 전처리 정의
transforms = Compose([
   RandomCrop((32, 32), padding=4),
   RandomHorizontalFlip(p=0.5),
   ToTensor(),
   Normalize(mean=(0.4914, 0.4822, 0.4465),
             std=(0.247, 0.243, 0.261))
])

# 교사모델 학습에 필요한 요소 정의

In [3]:
# 학습용 데이터 준비
training_data = CIFAR10(root="./",
                        train=True,
                        download=True,
                        transform=transforms)
test_data = CIFAR10(root="./",
                    train=False,
                    download=True,
                    transform=transforms)

# 검증용 데이터 준비
train_loader = DataLoader(
    training_data,
    batch_size=32,
    shuffle=True)
test_loader = DataLoader(
    test_data,
    batch_size=32,
    shuffle=False)


device = "cuda" if torch.cuda.is_available() else "cpu"

# 교사 모델 정의
teacher = resnet34(pretrained=False, num_classes=10)
teacher.to(device)

lr = 1e-5
optim = Adam(teacher.parameters(), lr=lr)

100%|██████████| 170498071/170498071 [00:03<00:00, 43445504.73it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# 교사모델 학습 루프 정의

In [5]:
# 학습 루프
for epoch in range(30):
   iterator = tqdm.tqdm(train_loader)
   for data, label in iterator:
       optim.zero_grad()

       preds = teacher(data.to(device))

       loss = nn.CrossEntropyLoss()(preds, label.to(device))
       loss.backward()
       optim.step()

       iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")

# 교사 모델의 가중치 저장
torch.save(teacher.state_dict(), "teacher.pth")

epoch:30 loss:1.0898953676223755: 100%|██████████| 1563/1563 [01:09<00:00, 22.64it/s]


# 교사 모델 성능 평가하기

In [7]:
# 교사 모델의 가중치 불러오기
teacher.load_state_dict(torch.load("/content/teacher.pth", map_location=device))

num_corr = 0

# 교사 모델의 성능 검증
with torch.no_grad():
   for data, label in test_loader:

       output = teacher(data.to(device))
       preds = output.data.max(1)[1]
       corr = preds.eq(label.to(device).data).sum().item()
       num_corr += corr

   print(f"Accuracy:{num_corr/len(test_data)}")

Accuracy:0.6184


# 데이터 생성자 정의

In [8]:
import torch.nn.functional as F

class Generator(nn.Module):
   def __init__(self, dims=256, channels=3):
       super(Generator, self).__init__()

       # 256 차원 벡터를 입력받아 128채널 8X8 이미지 생성
       self.l1 = nn.Sequential(nn.Linear(dims, 128 * 8 * 8))

       self.conv_blocks0 = nn.Sequential(
           nn.BatchNorm2d(128),
       )
       self.conv_blocks1 = nn.Sequential(
           nn.Conv2d(128, 128, 3, stride=1, padding=1),
           nn.BatchNorm2d(128),
           nn.LeakyReLU(0.2),  # ① 활성화 함수
       )
       self.conv_blocks2 = nn.Sequential(
           nn.Conv2d(128, 64, 3, stride=1, padding=1),
           nn.BatchNorm2d(64),
           nn.LeakyReLU(0.2),
           nn.Conv2d(64, channels, 3, stride=1, padding=1),
           nn.Tanh(),
           nn.BatchNorm2d(channels, affine=False)  # ② 배치 정규화
       )
       # affin 인수는 편향의 유무를 결정합니다.

   def forward(self, z):
       # 256차원 벡터를 128채널 8X8 이미지로 변환
       out = self.l1(z.view(z.shape[0], -1))
       out = out.view(out.shape[0], -1, 8, 8)

       out = self.conv_blocks0(out)
       # ③ 이미지를 두 배로 늘려줌
       out = nn.functional.interpolate(out, scale_factor=2)
       out = self.conv_blocks1(out)
       out = nn.functional.interpolate(out, scale_factor=2)
       out = self.conv_blocks2(out)
       return out
       # interpolate(out, scale_factor)
       # out이 scale_factor배가 되도록 변환합니다, 이미지 크기가 커지면서 생기는 빈 공간을 자동으로 보관합니다.

# 학생모델과 생성자 학습
1. 먼저 앞서 학습한 교사 모델을 불러옵니다.
2. 다음으로 학생 모델을 정의합니다.
3. 데이터를 만들어줄 생성자를 정의하고
4. 최적화 기법을 정의합니다.

In [11]:
from torch.optim.sgd import SGD

# ❶ 교사 모델 불러오기
teacher = resnet34(pretrained=False, num_classes=10)
teacher.load_state_dict(torch.load("/content/teacher.pth", map_location=device))
teacher.to(device)
teacher.eval()

# ❷ 학생 모델 정의
student = resnet18(pretrained=False, num_classes=10)
student.to(device)

# ❸ 생성자 정의
generator = Generator()
generator.to(device)

# ❹ 생성자는 Adam으로, 학생 모델은 SGD를 이용해서 학습
G_optim = Adam(generator.parameters(), lr=1e-3)
S_optim = SGD(student.parameters(), lr=0.1, weight_decay=5e-4, momentum=0.9)
# SGD(params, lr)
# 모델의 가중치 params를 학습률 lr을 이용해 경사 하강법으로 최적화 합니다.

# 학습 루프 정의
- 학생 모델을 5번 학습하고 생성 모델을 한 번 학습합니다.
생성 모델은 한 번 학습할 때마다 분류하기 더 어려운 이미지를 만들어냅니다.
그렇기 때문에 생성자가 너무 빨리 학습되면 학생 모델을 학습할 수 없습니다.

In [12]:
for epoch in range(500):
    # ⓵학생 모델을 5번, 생성자는 1번 가중치를 학습
    for _ in range(5):
        # ❶이미지 생성을 위한 노이즈 생성
        noise = torch.randn(256, 256, 1, 1, device=device)
        S_optim.zero_grad()
        # ❷이미지 생성
        fake = generator(noise).detach()
        # detach() : 텐서를 계산 그래프로부터 떼어냅니다.
        # 오차를 역전파할 때 detach된 텐서의 기울기는 역전파되지 않습니다.
        # ❸교사의 예측
        teacher_output = teacher(fake)
        # ❹학생의 예측
        student_output = student(fake)
        # ❺학생의 오차 계산
        S_loss = nn.L1Loss()(student_output, teacher_output.detach())

        print(f"epoch{epoch}: S_loss {S_loss}")
        # ➏ 오차 역전파
        S_loss.backward()
        S_optim.step()

    # 생성자 학습
    # 이미지 생성을 위한 특징 공간 상의 좌표 정의
    # 노이즈를 이용해 이미지를 생성
    # 생성자가 만들어낸 이미지를 이용해 교사와 학생 모델의 출력을 계산
    # 교사와 학생 모델의 차이를 생성자의 오차로 정의하고 오차를 역전파합니다.
    # ➊ 이미지 생성을 위한 노이즈 정의
    noise = torch.randn(256, 256, 1, 1, device=device)
    G_optim.zero_grad()
    # ➋ 이미지 생성
    fake = generator(noise)

    # ➌ 교사와 학생 모델의 출력 계산
    teacher_output = teacher(fake)
    student_output = student(fake)

    # ➍ 생성자의 오차 계산
    G_loss = -1 * nn.L1Loss()(student_output, teacher_output)

    # ➎ 오차 역전파
    G_loss.backward()
    G_optim.step()

    print(f"epoch{epoch}: G_loss {G_loss}")

epoch0: S_loss 1.9717082977294922
epoch0: S_loss 1.2519584894180298
epoch0: S_loss 1.7564258575439453
epoch0: S_loss 1.489790439605713
epoch0: S_loss 1.322715401649475
epoch0: G_loss -1.5480488538742065
epoch1: S_loss 1.8648728132247925
epoch1: S_loss 1.2519996166229248
epoch1: S_loss 1.7864760160446167
epoch1: S_loss 1.5815399885177612
epoch1: S_loss 1.386370062828064
epoch1: G_loss -1.6486848592758179
epoch2: S_loss 1.8304184675216675
epoch2: S_loss 1.4478466510772705
epoch2: S_loss 1.4819988012313843
epoch2: S_loss 1.5072094202041626
epoch2: S_loss 1.1870015859603882
epoch2: G_loss -1.4121688604354858
epoch3: S_loss 1.7197579145431519
epoch3: S_loss 1.2487398386001587
epoch3: S_loss 1.4621390104293823
epoch3: S_loss 1.2709906101226807
epoch3: S_loss 1.1388708353042603
epoch3: G_loss -1.1003386974334717
epoch4: S_loss 1.3414746522903442
epoch4: S_loss 1.074593186378479
epoch4: S_loss 1.2056077718734741
epoch4: S_loss 0.9659357070922852
epoch4: S_loss 1.065073013305664
epoch4: G_loss 

# 학생 모델 성능 평가하기

In [ ]:
num_corr = 0

student.load_state_dict(
    torch.load("./student.pth", map_location=device))

# 학습용 데이터에 대한 정확도
with torch.no_grad():
    for data, label in train_loader:

        output = student(data.to(device))
        preds = output.data.max(1)[1]
        corr = preds.dq(label.to(device).data).sum().item()
        num_corr += corr

    print(f"Accuracy:{num_corr/len(training_data)}")

num_corr = 0

# 검증용 데이터에 대한 정확도
with torch.no_grad():
    for data, label in test_loader:

        output = student(data.to(device))
        preds = output.data.max(1)[1]
        corr = preds.eq(label.to(device).data).sum().item()
        num_corr += corr

    print(f"Accuracy:{num_corr/len(test_data)}")